# Preprocesamiento de texto NLP SIN USAR LIBRERIAS
**By Jean Carlo Alvarez Ramirez**

En la demo vamos a ver:

Limpieza, la remoción del contenido no deseado

Normalización, la conversión diferentes formas a una sola

Tokenización, la separación del texto en tókenes (unidades mínimas, por ejemplo palabras)

Separación en conjuntos de datos: entrenamiento, validación, prueba

Generación del vocabulario, la lista de tókenes conocidos

Creacion de embeddings

# Importar librerias

In [2]:
import re
import string
import random
import numpy as np
from sklearn.model_selection import train_test_split

**import re**
Importa el módulo re que proporciona funciones para trabajar con expresiones regulares en Python

**¿Para qué sirve?**
Permite realizar tareas como búsqueda, coincidencia y manipulación de cadenas de texto utilizando patrones

**import string**
Importa el módulo string que contiene constantes y funciones relacionadas con la manipulación de cadenas de texto

**¿Para qué sirve?**
Ofrece listas predefinidas de caracteres comunes, como letras, dígitos y símbolos

**import random**
Importa el módulo random que permite generar números aleatorios y realizar selecciones al azar

**¿Para qué sirve?**
Es útil para tareas como crear datos aleatorios, barajar listas o realizar simulaciones

**import numpy as np**
Importa la librería numpy y la renombra como np por convención

**¿Para qué sirve?**
Es una librería clave para cálculos científicos y operaciones con matrices y arreglos multidimensionales

**from sklearn.model_selection import train_test_split**

Importa la función train_test_split desde el módulo sklearn.model_selection

**¿Para qué sirve?**

Divide un conjunto de datos en subconjuntos de entrenamiento y prueba para entrenar modelos de Machine Learning

**Parámetros clave:**
- X: Características (features) de entrada
- y: Etiquetas (targets)
- test_size: Proporción de datos para prueba (ej: 0.2 usa el 20% para testing)
- random_state: Fija la semilla para resultados reproducibles
- shuffle: Si es True (por defecto), mezcla los datos antes de dividir

**¿Qué pasa si cambia algo?**
- test_size afecta el balance entre datos para entrenar y probar
- random_state garantiza resultados iguales en ejecuciones repetidas
- shuffle=False puede ser útil si los datos tienen un orden temporal

# Limpiar y Normalizar texto

La función limpiar_texto recibe un string y realiza varias operaciones:
Convierte el texto a minúsculas para unificar la representación.
Elimina dígitos y signos de puntuación utilizando expresiones regulares.
Quita espacios innecesarios al inicio y final del texto.




In [3]:
def limpiar_texto(texto):
    """
      - Convierte a minúsculas
      - Elimina dígitos
      - Elimina signos de puntuación
      - Elimina espacios extra al inicio y al final
    """
    texto = texto.lower()  # Normalizamos a minúsculas
    texto = re.sub(r'\d+', '', texto)  # Eliminamos dígitos
    # Eliminamos signos de puntuación utilizando la librería string
    texto = re.sub(r'[' + re.escape(string.punctuation) + ']', '', texto)
    texto = texto.strip()  # Quitamos espacios al inicio y al final
    return texto

**def limpiar_texto(texto):**

Define una función llamada limpiar_texto que recibe un parámetro texto, que debe ser una cadena de caracteres (str) esta se encarga de limpiar y normalizar el texto y devolverlo una vez limpio y normalizado.

**texto.lower()**
Convierte todos los caracteres de la cadena a minúsculas

**¿Por qué es útil?**
Evita problemas de sensibilidad entre mayúsculas y minúsculas al comparar textos

**re.sub(r'\d+', '', texto)** - **re.sub(pattern, repl, string)**

Reemplaza todas las coincidencias del patrón por la cadena repl

- r'\d+' Es una expresión regular que:

    - \d coincide con cualquier dígito (0-9)
    - "+" significa uno o más dígitos consecutivos.
    - '' Indica que se reemplazan los dígitos encontrados por una cadena vacía, es decir, se eliminan

**re.sub(r'[' + re.escape(string.punctuation) + ']', '', texto)**

Crea un patrón que coincide con cualquiera de los signos de puntuación.

**¿Qué hace la línea?**
Elimina todos los signos de puntuación del texto

- string.punctuation : Contiene todos los signos de puntuación estándar ('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')

- re.escape(string.punctuation) : Añade barras invertidas a los caracteres especiales para que sean interpretados literalmente por la expresión regular

**texto.strip()**

Elimina espacios en blanco al inicio y al final de la cadena.
También elimina caracteres como \n (saltos de línea) o \t (tabulaciones).


# Tokenizar

Separa el texto limpio en tokens (palabras) utilizando la función split(), que separa por espacios.
En aplicaciones reales se pueden usar tokenizadores más robustos (por ejemplo, NLTK o spaCy).

In [4]:
def tokenizar_texto(texto):
    """
    Separa el texto en tokens utilizando la separación por espacios.
    En casos reales se podría usar una librería especializada (NLTK, spaCy, etc.)
    """
    tokens = texto.split()
    return tokens

**def tokenizar_texto(texto):**

Define una función llamada tokenizar_texto que recibe un parámetro texto, que debe ser una cadena de caracteres (str) el mismo se encarga de realizar la tokenizacion y devolver el texto en tokens.

**texto.split()**
El método split() divide la cadena en una lista de palabras, usando los espacios en blanco como separador por defecto

**¿Qué hace exactamente?**

- Cada vez que encuentra un espacio, crea un nuevo elemento en la lista
- También considera tabulaciones (\t) y saltos de línea (\n) como separadores
- Elimina automáticamente múltiples espacios consecutivos y los trata como uno solo

**¿Qué debes tener en cuenta?**

La función básica no elimina signos de puntuación. Por ejemplo:

El token "mundo!" mantiene el signo de exclamación
Si necesitas eliminar la puntuación antes de tokenizar, puedes usar la función limpiar_texto que revisamos antes

**¿Qué pasa si cambiamos algo?**

- Si usas split(',') en lugar de split(): Separará por comas en vez de espacios

- Si el texto tiene saltos de línea o tabulaciones: También serán tratados como separadores
.

# Creamos vocabulario

Se recorre el conjunto de entrenamiento para contar la frecuencia de cada token.
Se crea un diccionario que asigna un identificador numérico a cada token.
Se incluyen tokens especiales:

- PAD con valor 0 para representar padding

- UNK para tokens desconocidos que no aparecen en el vocabulario

Se puede establecer un parámetro min_freq para filtrar tokens poco frecuentes

In [5]:
def generar_vocabulario(tokenized_texts, min_freq=1):
    """
    Generamos un diccionario que mapea cada token a un número entero
    Se pueden aplicar filtros de frecuencia mínima para eliminar tokens raros
    Se agregan tokens especiales: <PAD> para padding y <UNK> para tokens desconocidos.
    """
    token_freq = {}
    for tokens in tokenized_texts:
        for token in tokens:
            token_freq[token] = token_freq.get(token, 0) + 1

    # Inicializamos el vocabulario con tokens especiales
    vocab = {"<PAD>": 0}  # 0 se reserva para el padding
    next_id = 1
    vocab["<UNK>"] = next_id  # Token para palabras desconocidas
    next_id += 1

    # Agregamos tokens que cumplen con la frecuencia mínima
    for token, freq in token_freq.items():
        if freq >= min_freq:
            vocab[token] = next_id
            next_id += 1

    return vocab

**def generar_vocabulario(tokenized_texts, min_freq=1):**

Define una función llamada generar_vocabulario que:


- Recibe tokenized_texts (una lista de listas de tokens)
- Usa min_freq como frecuencia mínima para incluir un token en el vocabulario (por defecto es 1)

**token_freq = {}**

Inicializa un diccionario vacío para almacenar la frecuencia de cada token

**for tokens in tokenized_texts:**
Itera sobre cada lista de tokens (cada "oración" o "texto" ya tokenizado)

**for token in tokens:**
Itera sobre cada token individual dentro de la lista actual

**token_freq.get(token, 0) + 1**

- token_freq.get(token, 0) obtiene la frecuencia actual del token o usa 0 si el token no existe aún en el diccionario

- Suma 1 para contar el token actual

Al final de este bucle, token_freq contendrá la cantidad de veces que aparece cada token en el conjunto de textos

**vocab = {"<PAD>": 0}**
Crea un diccionario vocab donde:

- "PAD" está mapeado al índice 0 (por convención en muchas redes neuronales para hacer padding)

- next_id = 1 : Establece el siguiente ID disponible para otros tokens

**vocab["UNK"] = next_id**
- Asigna el índice 1 al token especial "UNK", que se usará para palabras desconocidas

- next_id += 1:
Incrementa el contador para asignar IDs únicos a los siguientes tokens

**for token, freq in token_freq.items():**

Itera sobre cada token y su frecuencia en token_freq.

**if freq >= min_freq:**

Verifica si la frecuencia del token es igual o superior a min_freq.

**vocab[token] = next_id**

Si el token pasa el filtro de frecuencia, se le asigna un ID único.

**next_id += 1**
Se incrementa el contador para el siguiente token

**¿Qué pasa si cambias algo?**

- min_freq=1 → Incluye todos los tokens (excepto los que nunca aparecen).
- min_freq=3 → Excluye tokens con baja frecuencia.
- Si cambias el índice de PAD o UNK:
Podría afectar modelos que dependen de esos valores específicos

# Crear Embeddings

Se genera un vector aleatorio (de dimensión 50 en este ejemplo) para cada token del vocabulario.
Estos vectores son útiles como representación de palabras en modelos de NLP. En la práctica se pueden usar embeddings preentrenados como Word2Vec o GloVe

In [6]:
def crear_embeddings(vocab, embedding_dim=50):
    """
    Creamos embedding para cada token del vocabulario
    Se generan vectores aleatorios de dimensión embedding_dim para ilustrar el concepto
    """
    embeddings = {}
    np.random.seed(42)  # Para reproducibilidad
    for token in vocab:
        embeddings[token] = np.random.rand(embedding_dim)
    return embeddings

**def crear_embeddings(vocab, embedding_dim=50):**

Define una función llamada crear_embeddings que:

- vocab → es un diccionario que mapea tokens a números enteros (creado por la función generar_vocabulario)
- embedding_dim=50 → define la dimensión del vector de embedding (por defecto es 50)

**embeddings = {}**
Inicializa un diccionario vacío llamado embeddings que almacenará el token como clave y su vector como valor

**np.random.seed(42)**

Fija la semilla aleatoria de NumPy para que los resultados sean reproducibles

- Cada vez que ejecutes la función, obtendrás los mismos números aleatorios.
- Cambiar el valor (por ejemplo, a np.random.seed(99)) generaría resultados distintos

**for token in vocab:**
Itera sobre cada token en el vocabulario.

**np.random.rand(embedding_dim)**
Genera un arreglo de números aleatorios entre 0 y 1 con tamaño igual a embedding_dim

**¿Qué pasa si cambias algo?**
- embedding_dim=100 → Genera vectores más largos con mayor capacidad de representación.
- np.random.seed() → Un valor diferente cambia los números aleatorios

# Ejecutamos

- Se define una lista de oraciones de ejemplo
- Se aplican sucesivamente las funciones de limpieza, tokenización, división en conjuntos y generación del vocabulario
- Se generan embeddings
- See imprimen los resultados en consola para visualizar cada paso del pre-procesamiento

In [7]:
# Ejemplo de datos: lista de oraciones en español
textos = [
        "¡Hola! Chicos y chicas este es un ejemplo de pre-procesamiento de texto",
        "El pre-procesamiento incluye la limpieza, normalización, tokenización, y generacion de embeddings",
        "También se puede realizar este ejercicio pero usando librerias",
        "Espero este ejercicio les sea de mucha utilidad para comprender el prerpocesamiento.",
        "La creación de embeddings es opcional, pero muy útil para NLP."
    ]

Se creo un arreglo con oraciones

In [8]:
# Limpieza y Normalización
textos_limpiados = [limpiar_texto(texto) for texto in textos]

Ejecuto la funcion **limpiar_texto** y le paso los textos uno por uno a la funcion gracias a la iteracion, estos textos limpios los almaceno en la variable t**extos_limpiados**

In [9]:
# Tokenización
textos_tokenizados = [tokenizar_texto(texto) for texto in textos_limpiados]

Ejecuto la funcion **tokenizar_texto** y le paso los textos ya limpiados uno por uno a la funcion gracias a la iteracion, estos textos tokenizados los almaceno en la variable **ttextos_tokenizados**

In [10]:
# Separación en conjuntos de datos
# Utilizamos train_test_split para dividir el dataset en entrenamiento, validación y prueba
# En este ejemplo, el conjunto de datos es pequeño; en aplicaciones reales se trabaja con muchos datos
train_texts, temp_texts = train_test_split(textos_tokenizados, test_size=0.4, random_state=42)
val_texts, test_texts = train_test_split(temp_texts, test_size=0.5, random_state=42)

**train_texts, temp_texts = train_test_split(textos_tokenizados, test_size=0.4, random_state=42)**

Usa la función train_test_split de sklearn.model_selection para dividir los datos en:

- Conjunto de entrenamiento (train_texts)
- Conjunto temporal (temp_texts) que será dividido más adelante en validación y prueba

**Parámetros:**
- textos_tokenizados → Lista de textos ya tokenizados (listas de tokens).
- test_size=0.4 → El 40% de los datos se asigna al conjunto temporal -(temp_texts) y el 60% al conjunto de entrenamiento (train_texts)
- random_state=42 → Fija la semilla aleatoria para asegurar resultados reproducibles.
(Si cambias el valor, obtendrás una división diferente)

**val_texts, test_texts = train_test_split(temp_texts, test_size=0.5, random_state=42)**

Divide el conjunto temporal (temp_texts) en:

- Conjunto de validación (val_texts)
- Conjunto de prueba (test_texts)

Parámetros:
- temp_texts → Conjunto temporal generado en la división anterior.
- test_size=0.5 → Divide a la mitad el conjunto temporal:
  - 50% va al conjunto de validación.
  - 50% al conjunto de prueba.
- random_state=42 → Asegura la reproducibilidad.



In [11]:
# Generación del vocabulario (usando solo el conjunto de entrenamiento)
vocab = generar_vocabulario(train_texts, min_freq=1)

Ejecuto la funcion **generar_vocabulario** pra crear el vocabulario de acuerdo al set de datos de entrenemiento, este vocabulario lo guardo en la varibale **vocab**

In [12]:
# Creación de embeddings (opcional)
embeddings = crear_embeddings(vocab, embedding_dim=50)

Ejectuto la funcion de **crear_embeddings** a la cual le paso el vocabulario generado para que construya y cree los embeddings de cada palabra del vocabulario en un tamaño en especifico en este caso en un embedding de 50 dimensiones y guardo los embedding creados en la varibale **embeddings**

# Resultados

In [13]:
print("Textos Originales:")
for t in textos:
  print("  ", t)

Textos Originales:
   ¡Hola! Chicos y chicas este es un ejemplo de pre-procesamiento de texto
   El pre-procesamiento incluye la limpieza, normalización, tokenización, y generacion de embeddings
   También se puede realizar este ejercicio pero usando librerias
   Espero este ejercicio les sea de mucha utilidad para comprender el prerpocesamiento.
   La creación de embeddings es opcional, pero muy útil para NLP.


In [14]:
print("\nTextos Limpios y Normalizados:")
for t in textos_limpiados:
    print("  ", t)


Textos Limpios y Normalizados:
   ¡hola chicos y chicas este es un ejemplo de preprocesamiento de texto
   el preprocesamiento incluye la limpieza normalización tokenización y generacion de embeddings
   también se puede realizar este ejercicio pero usando librerias
   espero este ejercicio les sea de mucha utilidad para comprender el prerpocesamiento
   la creación de embeddings es opcional pero muy útil para nlp


In [15]:
print("\nTokenización:")
for tokens in textos_tokenizados:
    print("  ", tokens)


Tokenización:
   ['¡hola', 'chicos', 'y', 'chicas', 'este', 'es', 'un', 'ejemplo', 'de', 'preprocesamiento', 'de', 'texto']
   ['el', 'preprocesamiento', 'incluye', 'la', 'limpieza', 'normalización', 'tokenización', 'y', 'generacion', 'de', 'embeddings']
   ['también', 'se', 'puede', 'realizar', 'este', 'ejercicio', 'pero', 'usando', 'librerias']
   ['espero', 'este', 'ejercicio', 'les', 'sea', 'de', 'mucha', 'utilidad', 'para', 'comprender', 'el', 'prerpocesamiento']
   ['la', 'creación', 'de', 'embeddings', 'es', 'opcional', 'pero', 'muy', 'útil', 'para', 'nlp']


In [16]:
print("\nVocabulario Generado:")
print(vocab)


Vocabulario Generado:
{'<PAD>': 0, '<UNK>': 1, 'también': 2, 'se': 3, 'puede': 4, 'realizar': 5, 'este': 6, 'ejercicio': 7, 'pero': 8, 'usando': 9, 'librerias': 10, '¡hola': 11, 'chicos': 12, 'y': 13, 'chicas': 14, 'es': 15, 'un': 16, 'ejemplo': 17, 'de': 18, 'preprocesamiento': 19, 'texto': 20, 'espero': 21, 'les': 22, 'sea': 23, 'mucha': 24, 'utilidad': 25, 'para': 26, 'comprender': 27, 'el': 28, 'prerpocesamiento': 29}


In [20]:
# Ejemplo: mostrar el embedding de la palabra "preprocesamiento"
print("\nEmbedding para la palabra 'preprocesamiento':")
# Notar que en nuestro proceso de limpieza no se conserva la tilde o signo,
# y la palabra se tokeniza como 'preprocesamiento' (sin guion ni tilde)
if "preprocesamiento" in embeddings:
      print(embeddings["preprocesamiento"])
else:
      print("  El token 'preprocesamiento' no se encuentra en el vocabulario.")


Embedding para la palabra 'puede:
[0.64203165 0.08413996 0.16162871 0.89855419 0.60642906 0.00919705
 0.10147154 0.66350177 0.00506158 0.16080805 0.54873379 0.6918952
 0.65196126 0.22426931 0.71217922 0.23724909 0.3253997  0.74649141
 0.6496329  0.84922341 0.65761289 0.5683086  0.09367477 0.3677158
 0.26520237 0.24398964 0.97301055 0.39309772 0.89204656 0.63113863
 0.7948113  0.50263709 0.57690388 0.49251769 0.19524299 0.72245212
 0.28077236 0.02431597 0.6454723  0.17711068 0.94045858 0.95392858
 0.91486439 0.3701587  0.01545662 0.92831856 0.42818415 0.96665482
 0.96361998 0.85300946]
